In [5]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from statsmodels.nonparametric.smoothers_lowess import lowess

from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy import signal


* Este método es usado para dar inicio al algoritmo.
* @param datesPath:string   Este primer parámetro corresponde al path 
* absoluto del dataset de fechas para las que se desea predecir la TRM
**/


def predict_trm(datesPath):
    ...
    return outPath

trm_raw = pd.read_excel('1.1.1.TCM_Serie histórica IQY.xlsx')
# TRM
# frop unwanted header and useless columns
trm_clean = trm_raw.drop(trm_raw.index[:6], axis=0).drop(['Unnamed: 2','Unnamed: 3'],axis=1)
trm_cols = trm_clean.iloc[0].values
trm_clean.columns = trm_cols
# use first entry as new column names
trm_clean.drop(trm_clean.index[0], inplace=True) 
trm_clean = trm_clean.rename(columns = {'Fecha (dd/mm/aaaa)':'fecha',
                                        'Tasa de cambio representativa del mercado (TRM)':'trm'})
# rest index
trm_clean = trm_clean.reset_index().drop('index', axis=1)
# drop unwanted disclaimers
trm_clean = trm_clean.iloc[trm_clean.index[:-4]] 
trm_clean['fecha'] = pd.to_datetime(trm_clean['fecha'],format='%Y/%m/%d')
trm_clean['trm'] = trm_clean['trm'].astype(float)
#sacando features con tiempo
trm_clean['year'] = trm_clean['fecha'].dt.year
trm_clean['month'] = trm_clean['fecha'].dt.month
trm_clean['day'] = trm_clean['fecha'].dt.day
trm_clean['weekday'] = trm_clean['fecha'].dt.weekday
trm_clean['is_weekday'] = 0
trm_clean.loc[trm_clean['weekday']< 5,'is_weekday'] = 1
pd_trm = trm_clean.set_index('fecha')
pd_data = pd_trm.copy()
pd_data = pd_data.rename(columns={'trm':'value'})



forecast_index = pd.read_csv('test_file.txt', header=None)
forecast_index[0] = pd.to_datetime(forecast_index[0])
forecast_index['month'] = forecast_index[0].dt.month
forecast_index

order = (1, 2, 2) 
#seasonal order goes AR, D MA,SEASON LENGHT
# D SHOULD BE GIVEN BY THE CH TEST OR GRID SEARCH SAME AS THE OTHER
seasonal_oder =  (0, 0, 0, 12)
model = SARIMAX(pd_data.value ,exog = pd_data.month ,order=order,seasonal_order=seasonal_oder,freq='D').fit()

test_pred = model.forecast(len(forecast_index.month),exog = forecast_index.month ,alpha=0.05)  # 95% conf
test_pred.to_csv('output.txt', sep=',', index=False,header=False)


In [9]:
def predict_trm(dates_path):
    trm_raw = pd.read_excel('1.1.1.TCM_Serie histórica IQY.xlsx')
    # TRM
    # frop unwanted header and useless columns
    trm_clean = trm_raw.drop(trm_raw.index[:6], axis=0).drop(['Unnamed: 2','Unnamed: 3'],axis=1)
    trm_cols = trm_clean.iloc[0].values
    trm_clean.columns = trm_cols
    # use first entry as new column names
    trm_clean.drop(trm_clean.index[0], inplace=True) 
    trm_clean = trm_clean.rename(columns = {'Fecha (dd/mm/aaaa)':'fecha',
                                            'Tasa de cambio representativa del mercado (TRM)':'trm'})
    # rest index
    trm_clean = trm_clean.reset_index().drop('index', axis=1)
    # drop unwanted disclaimers
    trm_clean = trm_clean.iloc[trm_clean.index[:-4]] 
    trm_clean['fecha'] = pd.to_datetime(trm_clean['fecha'],format='%Y/%m/%d')
    trm_clean['trm'] = trm_clean['trm'].astype(float)
    #sacando features con tiempo
    trm_clean['year'] = trm_clean['fecha'].dt.year
    trm_clean['month'] = trm_clean['fecha'].dt.month
    trm_clean['day'] = trm_clean['fecha'].dt.day
    trm_clean['weekday'] = trm_clean['fecha'].dt.weekday
    trm_clean['is_weekday'] = 0
    trm_clean.loc[trm_clean['weekday']< 5,'is_weekday'] = 1
    pd_trm = trm_clean.set_index('fecha')
    pd_data = pd_trm.copy()
    pd_data = pd_data.rename(columns={'trm':'value'})
    
    #create forecasting for dates
    #current_date  = pd.to_datetime('today')
    #end_date = pd.to_datetime('2020-04-21')
    pd_t_series = pd_data.loc['2010-01-01':].copy()
    forecast_index = pd.read_csv(dates_path, header=None)
    forecast_index[0] = pd.to_datetime(forecast_index[0])
    forecast_index['month'] = forecast_index[0].dt.month

    order = (1, 2, 2) 
    #seasonal order goes AR, D MA,SEASON LENGHT
    # D SHOULD BE GIVEN BY THE CH TEST OR GRID SEARCH SAME AS THE OTHER
    seasonal_oder =  (0, 0, 0, 12)
    model = SARIMAX(pd_data.value ,exog = pd_data.month ,order=order,seasonal_order=seasonal_oder,freq='D').fit()

    test_pred = model.forecast(len(forecast_index.month),exog = forecast_index.month ,alpha=0.05)  # 95% conf
    pred_df = pd.DataFrame(test_pred,columns=['prediction'])
    pred_df['date'] = pred_df.index
    pred_df.to_csv('output.txt', sep=',', index=None,header=None,mode='a')
    
    return pred_df


In [10]:
ptest = predict_trm('test_file.txt')

/Users/rasputin/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [12]:
ptest

,prediction,date
2020-03-14,4058.851327,2020-03-14
2020-03-15,4054.343259,2020-03-15
2020-03-16,4057.003509,2020-03-16
2020-03-17,4057.873311,2020-03-17
2020-03-18,4059.190317,2020-03-18
2020-03-19,4060.395624,2020-03-19
2020-03-20,4061.628831,2020-03-20
2020-03-21,4062.855069,2020-03-21
2020-03-22,4064.083047,2020-03-22
2020-03-23,4065.310591,2020-03-23
